In [5]:
from pathlib import Path

import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from bs4 import BeautifulSoup


import libs.commons as commons

In [6]:
train_path = Path(commons.dataset_path) / "train.csv"

train_set = pd.read_csv(train_path)

In [7]:
train_set['text_len'] = train_set['text'].apply(len)

In [8]:
train_set.head()

,id,keyword,location,text,target,text_len
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88


In [9]:
## Frequency of Keyword values
# train_set.describe()
# train_set['keyword'].nunique()
key_group = train_set.groupby('keyword')
key_group.count()

,id,location,text,target,text_len
keyword,,,,,
ablaze,36,29,36,36,36
accident,35,28,35,35,35
aftershock,34,25,34,34,34
airplane%20accident,35,29,35,35,35
ambulance,38,26,38,38,38
...,...,...,...,...,...
wounded,37,27,37,37,37
wounds,33,27,33,33,33
wreck,37,27,37,37,37


In [10]:
## Frequency of Keyword values
loc_group = train_set.groupby('location')
loc_group.count().sort_values(by='id', ascending=False)

,id,keyword,text,target,text_len
location,,,,,
USA,104,104,104,104,104
New York,71,71,71,71,71
United States,50,50,50,50,50
London,45,45,45,45,45
Canada,29,29,29,29,29
...,...,...,...,...,...
Hueco Mundo,1,1,1,1,1
"Hughes, AR",1,1,1,1,1
"Huntington, WV",1,1,1,1,1


In [11]:
def clean_loc(x):
    x = str(x)
    if x:
        return x.lower()
    else:
        return None

train_proc = train_set.copy()
train_proc['location_clean'] = train_proc['location'].apply(clean_loc)
loc_group = train_proc.groupby('location_clean')
loc_group.count().sort_values(by='id', ascending=False)

,id,keyword,location,text,target,text_len
location_clean,,,,,,
nan,2533,2472,0,2533,2533,2533
usa,104,104,104,104,104,104
new york,75,75,75,75,75,75
united states,50,50,50,50,50,50
london,49,49,49,49,49,49
...,...,...,...,...,...,...
"haiku, maui, hawaii",1,1,1,1,1,1
hailing from dayton,1,1,1,1,1,1
halfrica,1,1,1,1,1,1


In [12]:
for loc in loc_group.groups.keys():
    print(loc)

o be, yet
nottingham
nottingham, england
nottingham, united kingdom
notts
nova scotia, canada
novi, mi
nowhere
nowhere islands/smash manor
nowhere. everywhere.
numa casa de old yellow bricks
numenor
nunya
nv
ny
ny capital district
ny || live easy? 
ny, ct &amp; greece
ny, ny
nyc
nyc / international
nyc :) ex- #islamophobe
nyc area
nyc metro
nyc&amp;nj
nyc, new york
nyc,us - cali, colombia
nyc-la-miami
nyhc
oakland
oakland, ca
oblivion?
ocean city, nj
odawara, japan
oes 4th point. sisstar &amp; ti
official website
ogba, lagos, nigeria
ohio
ohio, usa
ojodu,lagos
ok
okanagan valley, bc
oklahoma
oklahoma city
oklahoma city, ok
oklahoma, usa
okuma town, fukushima
olathe, ks
old blighty
oldenburg // london
olympia, wa
oman muscat al seeb 
on
on a beach 
on a catwalk somewhere
on the court 
on the go
on the toilet having a dump 
on the web
on to the next adventure
on twitter
on twitter 
ona block w/ my boy ??
ondo
one world
oneonta, ny/ staten island, ny
online 
online 24/7. not even kidding.

In [13]:
# Preprocess and clean text features
nltk.download("stopwords")

stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sampaio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:

def process_text(text):
    text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9#]", " ", text) # Keep only alphanumeric characters and hashtags
    text = text.lower()
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    # words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [37]:
for entry in train_proc.loc[0:20, 'text']:
    print(process_text(entry))


[&#39;deeds&#39;, &#39;reason&#39;, &#39;#earthquake&#39;, &#39;may&#39;, &#39;allah&#39;, &#39;forgive&#39;, &#39;us&#39;]
[&#39;forest&#39;, &#39;fire&#39;, &#39;near&#39;, &#39;la&#39;, &#39;ronge&#39;, &#39;sask&#39;, &#39;canada&#39;]
[&#39;residents&#39;, &#39;asked&#39;, &#39;shelter&#39;, &#39;place&#39;, &#39;notified&#39;, &#39;officers&#39;, &#39;evacuation&#39;, &#39;shelter&#39;, &#39;place&#39;, &#39;orders&#39;, &#39;expected&#39;]
[&#39;13&#39;, &#39;000&#39;, &#39;people&#39;, &#39;receive&#39;, &#39;#wildfires&#39;, &#39;evacuation&#39;, &#39;orders&#39;, &#39;california&#39;]
[&#39;got&#39;, &#39;sent&#39;, &#39;photo&#39;, &#39;ruby&#39;, &#39;#alaska&#39;, &#39;smoke&#39;, &#39;#wildfires&#39;, &#39;pours&#39;, &#39;school&#39;]
[&#39;#rockyfire&#39;, &#39;update&#39;, &#39;california&#39;, &#39;hwy&#39;, &#39;20&#39;, &#39;closed&#39;, &#39;directions&#39;, &#39;due&#39;, &#39;lake&#39;, &#39;county&#39;, &#39;fire&#39;, &#39;#cafire&#39;, &#39;#wildfires&#39;]
[&

In [47]:

def process_location(text):
    # print(text)
    text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9#]", " ", text) # Keep only alphanumeric characters and hashtags
    text = text.lower()
    words = text.split() # Split string into words
    # words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    # words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [65]:
train_not_missing = train_proc.dropna(axis=0, how='any', subset=['location']).reset_index(drop=True)
for entry in train_not_missing.loc[0:20, 'location']:
    print(process_location(entry))


[&#39;birmingham&#39;]
[&#39;est&#39;, &#39;september&#39;, &#39;2012&#39;, &#39;bristol&#39;]
[&#39;africa&#39;]
[&#39;philadelphia&#39;, &#39;pa&#39;]
[&#39;london&#39;, &#39;uk&#39;]
[&#39;pretoria&#39;]
[&#39;world&#39;, &#39;wide&#39;]
[&#39;paranaque&#39;, &#39;city&#39;]
[&#39;live&#39;, &#39;on&#39;, &#39;webcam&#39;]
[&#39;milky&#39;, &#39;way&#39;]
[&#39;greensboro&#39;, &#39;north&#39;, &#39;carolina&#39;]
[&#39;live&#39;, &#39;on&#39;, &#39;webcam&#39;]
[&#39;england&#39;]
[&#39;sheffield&#39;, &#39;township&#39;, &#39;ohio&#39;]
[&#39;india&#39;]
[&#39;barbados&#39;]
[&#39;anaheim&#39;]
[&#39;abuja&#39;]
[&#39;usa&#39;]
[&#39;south&#39;, &#39;africa&#39;]
[&#39;sao&#39;, &#39;paulo&#39;, &#39;brazil&#39;]
